# Loading the data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)

# Subcat 420201

There is a weekly data for subcategory 420201 of products. This sucategory was used to find the best model.

In [3]:
df = pd.read_parquet('/content/drive/MyDrive/weekly_data.parquet', engine='pyarrow')
df

,week_iso,index_material,index_store,qnt,qnt_loss,discount,action_mediana_price,reg_mediana_price,regular_price,action_price,netto,brutto,sum_action_price_by_brand,am_items_by_brand,count_park,count_cashzone,geolocal_type,format_merch,matrix_type,brand_id,shelf,country_id,type_for_customer,regular_pnz,promo_pnz,last_reg_mediana_price,last_action_mediana_price,type_bonus_id,bu_exists,flg_promo,is_holiday,qnt_by_brand
0,1,284,74,6,0.0,0.575235,11.90,28.015499,28.015499,11.90,90.0,90.0,332791.18750,59,0,4,mixed,C,P,998,730,RU,11,0,0,28.015499,11.90,0,0,1,1,121213
1,1,284,78,1,0.0,0.472078,14.79,28.015499,28.015499,14.79,90.0,90.0,389837.21875,59,0,4,mixed,C,P,998,730,RU,11,0,0,28.015499,14.79,0,0,1,1,121213
2,1,285,20,1,0.0,0.472757,14.22,26.970501,26.970501,14.22,90.0,90.0,389837.21875,59,0,20,mixed,H,P,998,730,RU,16,0,0,26.970501,14.22,0,0,1,1,121213
3,1,286,74,6,0.0,0.575592,11.89,28.015499,28.015499,11.89,90.0,90.0,332791.18750,59,0,4,mixed,C,P,998,730,RU,11,0,0,28.015499,11.89,0,0,1,1,121213
4,1,327,78,1,0.0,0.474287,13.08,24.880501,24.880501,13.08,88.0,88.0,271732.59375,59,0,4,mixed,C,P,998,730,RU,11,0,0,24.880501,13.08,0,0,1,1,121213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379532,52,2653,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19400,730,RU,0,0,0,NaN,NaN,0,0,0,0,0
379533,52,2654,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0
379534,52,2655,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0
379535,52,2656,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0


In [4]:
df.index_material.nunique()

269

In [ ]:
df.index_store.nunique()

83

In [ ]:
df.brand_id.nunique()

26

So, there are 269 unique products, 83 different shops and 26 brands in this subcategory.

## Baseline model **without** cannibalization effect for subcategory 420201

For the baseline model without cannibalization it was decided to use Linear Regression model.

In [5]:
# Splitting data on train/val/test (3 weeks in val and test)
def split_data(df):
    weeks = df['week_iso'].unique()
    weeks.sort()

    # Last 6 weeks - 3 for val and 3 for test
    val_weeks = weeks[-3:]
    test_weeks = weeks[-6:-3]
    train_weeks = weeks[:-6]

    # Разделяем данные
    train = df[df['week_iso'].isin(train_weeks)]
    val = df[df['week_iso'].isin(val_weeks)]
    test = df[df['week_iso'].isin(test_weeks)]

    return train, val, test

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [7]:
# 1. Function to create lag features
def add_lags(df, lags=3):
    """Add lagged sales quantities for each product-store combination"""
    df = df.sort_values(['index_material', 'index_store', 'week_iso'])
    for lag in range(1, lags+1):
        df[f'qnt_lag_{lag}'] = df.groupby(['index_material', 'index_store'])['qnt'].shift(lag)
    return df

# 2. Add lag features and split
df = add_lags(df.copy())
train, val, test = split_data(df)

# 3. Define feature types and target variable
target = 'qnt'
exclude_cols = ['week_iso', 'index_material', 'index_store', target]

# Numeric features (excluding lags which are already processed)
numeric_features = train.select_dtypes(include=['int16', 'int32', 'int64', 'float32', 'float64']).columns
numeric_features = [col for col in numeric_features if col not in exclude_cols and not col.startswith('qnt_lag_')]

# Categorical features
categorical_features = train.select_dtypes(include=['category', 'object']).columns.tolist()

# Sales lag features (already numeric)
lag_features = [f'qnt_lag_{i}' for i in range(1, 4)]

# All features to be used in model
all_features = numeric_features + categorical_features + lag_features

# 4. Create feature preprocessor with imputation
preprocessor = ColumnTransformer(
    transformers=[
        # Scale numeric features and lags (with imputation)
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features + lag_features),
        # One-hot encode categorical features (with imputation)
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

# 5. Create modeling pipeline
model = Pipeline([
    ('preprocessor', preprocessor),  # Feature preprocessing
    ('regressor', LinearRegression())  # Linear regression model
])

# 6. Prepare data (drop rows where target is NA)
train = train.dropna(subset=[target])
val = val.dropna(subset=[target])
test = test.dropna(subset=[target])

X_train, y_train = train[all_features], train[target]
X_val, y_val = val[all_features], val[target]
X_test, y_test = test[all_features], test[target]

# 7. Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['qnt_loss', 'discount',
                                                   'action_mediana_price',
                                                   'reg_mediana_price',
                                                   'regular_price',
                                                   'action_price', 'netto',
                                                   'brutto',
                                                   'sum_action_price_by_brand',
                                                   'count_park',
                                                   'count_cashzone', 'brand_id',
                                                   'shel..._pnz',
                                                   'promo_pnz',
                                                   'last_reg_mediana_price',
                                                   'last_action_mediana_price',
                                                   'qnt_by_brand', 'qnt_lag_1',
                                                   'qnt_lag_2', 'qnt_lag_3']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['geolocal_type',
                                                   'format_merch',
                                                   'matrix_type',
                                                   'country_id'])])),
                ('regressor', LinearRegression())])

In [8]:
# 8. Prediction function with post-processing
def predict(model, X):
    """Make predictions and apply post-processing"""
    y_pred = model.predict(X)
    y_pred = np.ceil(y_pred)  # Round up to nearest integer
    return np.maximum(y_pred, 0)  # Replace negatives with 0

# 9. Generate predictions
y_train_pred = predict(model, X_train)
y_val_pred = predict(model, X_val)
y_test_pred = predict(model, X_test)

# 10. Evaluation metrics
def print_metrics(y_true, y_pred, name):
    """Calculate and print evaluation metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Median Absolute Percentage Error (robust to zeros)
    median_ape = 100 * np.median(np.abs((y_true - y_pred) / (y_true + 1e-10)))

    print(f"\n{name} Metrics:")
    print(f"MAE: {mae:.2f} units")
    print(f"RMSE: {rmse:.2f} units")
    print(f"Median APE: {median_ape:.2f}%")

# Print metrics for all datasets
print_metrics(y_train, y_train_pred, "Train")
print_metrics(y_val, y_val_pred, "Validation")
print_metrics(y_test, y_test_pred, "Test")


Train Metrics:
MAE: 19.21 units
RMSE: 33.78 units
Median APE: 50.00%

Validation Metrics:
MAE: 17.76 units
RMSE: 28.46 units
Median APE: 43.59%

Test Metrics:
MAE: 17.46 units
RMSE: 29.14 units
Median APE: 43.75%


## The model **with** the cannibalization effect for subcategory 420201

The combination of clustering method and ML model that provided the best result for validation dataset was **OPTICS with Euclidean distance + LightGBM** model.

### OPTICS with Euclidean distance

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score
from sklearn.cluster import OPTICS
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [ ]:
def optics(data):

    # 1. Data aggregation for products
    product_features = data.groupby('index_material').agg({
        'qnt': 'mean',
        'action_mediana_price': 'mean',
        'reg_mediana_price': 'mean',
        'regular_price': 'mean',
        'action_price': 'mean',
        'netto': 'mean',
        'brutto': 'mean',
        'brand_id':'mean',
        'shelf':'mean',
        'sum_action_price_by_brand': 'mean',
        'am_items_by_brand': 'mean',
        'matrix_type': lambda x: x.mode()[0] if not x.mode().empty else pd.NA,
        'type_for_customer': lambda x: x.mode()[0] if not x.mode().empty else pd.NA,
        'bu_exists': lambda x: x.mode()[0] if not x.mode().empty else pd.NA
    }).reset_index()

    # 2. Handling missing values
    # For numeric features
    num_cols = ['qnt', 'action_mediana_price', 'reg_mediana_price',
               'regular_price', 'action_price', 'netto', 'brutto',
               'sum_action_price_by_brand', 'am_items_by_brand']
    product_features[num_cols] = product_features[num_cols].fillna(0)

    # For categories
    cat_cols = ['matrix_type', 'type_for_customer', 'bu_exists']
    for col in cat_cols:
        product_features[col] = product_features[col].astype(str).fillna('missing')
        product_features[col] = product_features[col].astype('category').cat.codes

    # 3. Scaler
    scaler = StandardScaler()
    X = scaler.fit_transform(product_features.drop('index_material', axis=1))

    # 4. Extra missing values handling
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    # 5. Clustering
    optics = OPTICS(
        min_samples=max(5, int(0.01*len(product_features))),  # Auto
        metric='euclidean',
        algorithm='ball_tree',
        n_jobs=-1,
        cluster_method='xi'
    )

    product_features['cluster'] = optics.fit_predict(X)

    # Silhouette score calculation
    unique_clusters = np.unique(product_features['cluster'])
    if len(unique_clusters) > 1:
        silhouette_avg = silhouette_score(X, product_features['cluster'])
        print(f"Silhouette Score: {silhouette_avg:.3f}")
    else:
        print("Silhouette score can not be calculated for 1 cluster")
        silhouette_avg = None

    # 6. Add cluster to the original data
    return data.merge(
        product_features[['index_material', 'cluster']],
        on='index_material',
        how='left'
    )['cluster'].fillna(-1).astype(int)

In [ ]:
# Apply
all_data = df.copy()
all_data['cluster'] = optics(all_data)
all_data

Silhouette Score: 0.275


,week_iso,index_material,index_store,qnt,qnt_loss,discount,action_mediana_price,reg_mediana_price,regular_price,action_price,netto,brutto,sum_action_price_by_brand,am_items_by_brand,count_park,count_cashzone,geolocal_type,format_merch,matrix_type,brand_id,shelf,country_id,type_for_customer,regular_pnz,promo_pnz,last_reg_mediana_price,last_action_mediana_price,type_bonus_id,bu_exists,flg_promo,is_holiday,qnt_by_brand,cluster
0,1,284,74,6,0.0,0.575235,11.90,28.015499,28.015499,11.90,90.0,90.0,332791.18750,59,0,4,mixed,C,P,998,730,RU,11,0,0,28.015499,11.90,0,0,1,1,121213,10
1,1,284,78,1,0.0,0.472078,14.79,28.015499,28.015499,14.79,90.0,90.0,389837.21875,59,0,4,mixed,C,P,998,730,RU,11,0,0,28.015499,14.79,0,0,1,1,121213,10
2,1,285,20,1,0.0,0.472757,14.22,26.970501,26.970501,14.22,90.0,90.0,389837.21875,59,0,20,mixed,H,P,998,730,RU,16,0,0,26.970501,14.22,0,0,1,1,121213,9
3,1,286,74,6,0.0,0.575592,11.89,28.015499,28.015499,11.89,90.0,90.0,332791.18750,59,0,4,mixed,C,P,998,730,RU,11,0,0,28.015499,11.89,0,0,1,1,121213,10
4,1,327,78,1,0.0,0.474287,13.08,24.880501,24.880501,13.08,88.0,88.0,271732.59375,59,0,4,mixed,C,P,998,730,RU,11,0,0,24.880501,13.08,0,0,1,1,121213,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379532,52,2653,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19400,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12
379533,52,2654,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12
379534,52,2655,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12
379535,52,2656,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.00000,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12


In [ ]:
all_data.cluster.value_counts()

,count
cluster,
-1,104251
5,40598
7,37107
18,31107
6,24381
4,23869
8,21877
16,18725
3,16024


There are 23 clusters.

### Adding cannibalization features

**Core idea:** It is assumed that products within the cluster affect the sales of each other (cannibaliation effect on each other). To catch this effect without identifying what product is the victim, the following features are created. These are te features that show the sales and the prices of OTHER products in the same cluster in the same shop and period. The function bellow adds such features.

**How it works:**

1. **Cluster isolation**:
   - For each product-store-week combination, it identifies "cluster mates" - other products that:
     - Belong to the same cluster/category
     - Are in the same store
     - Are sold in the same week
     - **Exclude** the current product itself

2. **Aggregate Statistics**:
   Computes three types of features for these cluster mates:
   - **Demand metrics**:
     - Sum of quantities sold (`cluster_qnt_other_sum`)
     - Median quantity sold (`cluster_qnt_other_median`)
   - **Price metrics**:
     - Mean action price (`cluster_price_other_mean`)
     - Median action price (`cluster_price_other_median`)
   - **Market saturation**:
     - Count of competing products (`cluster_other_items_count`)

3. **Temporal Dynamics** (Lags):
   Adds time-lagged versions (1, 2, 3 weeks back) of:
   - Total cluster mate sales (`cluster_other_qnt_sum_lag{lag}`)
   - Average cluster mate prices (`cluster_other_price_mean_lag{lag}`)

4. **Edge Case Handling**:
   - If no cluster mates exist:
     - Sets quantities to 0
     - Uses the focal product's own price as placeholder (assuming no competition)
     - Sets item count to 0

**Why this models cannibalization:**
- **Direct competition**: When `cluster_qnt_other_sum` is high, it is expected that the focal product's sales decrease (demand splitting)
- **Price pressure**: When `cluster_price_other_mean` is low (deep discounts), the focal product may lose sales unless it also discounts
- **Delayed effects**: Lagged features capture how past cluster activity influences current sales (e.g., customers stockpiling competing products)

In [ ]:
# Function to catch cannibalization effect
def add_cannibalization_features(df, lags=[1,2,3]):
    """
    Adds cannibalization features excluding the current product from the aggregation
    """
    # Create a copy without current product for each line
    df_list = []
    for idx, row in df.iterrows():
        # Filter: the same cluster and shop but not the same product
        cluster_mates = df[
            (df['cluster'] == row['cluster']) &
            (df['index_store'] == row['index_store']) &
            (df['index_material'] != row['index_material']) &
            (df['week_iso'] == row['week_iso'])
        ]

        # Aggregate
        agg_stats = cluster_mates.agg({
            'qnt': ['sum', 'median'],
            'action_price': ['mean', 'median']
        }).to_numpy().flatten()

        new_row = {
            'cluster_qnt_other_sum': agg_stats[0],
            'cluster_qnt_other_median': agg_stats[1],
            'cluster_price_other_mean': agg_stats[2],
            'cluster_price_other_median': agg_stats[3],
            'cluster_other_items_count': len(cluster_mates)
        }
        df_list.append(new_row)

    # Merge with the original data
    cannib_stats = pd.DataFrame(df_list, index=df.index)
    df = pd.concat([df, cannib_stats], axis=1)

    # Add lags
    df = df.sort_values(['index_material', 'index_store', 'week_iso'])
    for lag in lags:
        # Lags of aggregated features of outher products
        df[f'cluster_other_qnt_sum_lag{lag}'] = df.groupby(
            ['index_material', 'index_store']
        )['cluster_qnt_other_sum'].shift(lag)

        df[f'cluster_other_price_mean_lag{lag}'] = df.groupby(
            ['index_material', 'index_store']
        )['cluster_price_other_mean'].shift(lag)

    # Fill the missing values
    df.fillna({
        'cluster_qnt_other_sum': 0,
        'cluster_qnt_other_median': 0,
        'cluster_price_other_mean': df['action_price'],  # If there are no other products use its price
        'cluster_price_other_median': df['action_price'],
        'cluster_other_items_count': 0
    }, inplace=True)

    return df

In [ ]:
all_data = add_cannibalization_features(all_data)
all_data

,week_iso,index_material,index_store,qnt,qnt_loss,discount,action_mediana_price,reg_mediana_price,regular_price,action_price,netto,brutto,sum_action_price_by_brand,am_items_by_brand,count_park,count_cashzone,geolocal_type,format_merch,matrix_type,brand_id,shelf,country_id,type_for_customer,regular_pnz,promo_pnz,last_reg_mediana_price,last_action_mediana_price,type_bonus_id,bu_exists,flg_promo,is_holiday,qnt_by_brand,cluster,cluster_qnt_other_sum,cluster_qnt_other_median,cluster_price_other_mean,cluster_price_other_median,cluster_other_items_count,cluster_other_qnt_sum_lag1,cluster_other_price_mean_lag1,cluster_other_qnt_sum_lag2,cluster_other_price_mean_lag2,cluster_other_qnt_sum_lag3,cluster_other_price_mean_lag3
111500,18,278,25,26,0.0,0.021710,31.336395,32.480499,32.480499,31.775362,88.0,88.0,1.760417e+06,31,0,6,mixed,S,P,1049,720,RU,16,0,0,32.480499,31.336395,0,0,5,0,37385,0,0.0,0.0,31.775362,31.775362,0,NaN,NaN,NaN,NaN,NaN,NaN
118748,19,278,25,4,0.0,0.000000,32.389999,32.480499,32.480499,32.480499,88.0,88.0,3.856161e+05,31,0,6,mixed,S,P,1049,720,RU,16,0,0,32.480499,32.389999,0,0,2,0,51839,0,0.0,0.0,32.480499,32.480499,0,0.0,NaN,NaN,NaN,NaN,NaN
125946,20,278,25,22,0.0,0.004440,32.043037,32.110668,32.110668,31.970301,88.0,88.0,2.150233e+06,31,0,6,mixed,S,P,1049,720,RU,16,0,0,32.110668,32.043037,0,0,4,0,77133,0,20.0,0.0,20.000000,35.615501,1,0.0,NaN,0.0,NaN,NaN,NaN
18582,4,278,74,2,0.0,0.472342,17.690001,33.525501,33.525501,17.690001,88.0,88.0,2.075696e+05,31,0,4,mixed,C,P,1049,720,RU,11,0,0,33.525501,17.690001,0,0,1,0,54980,0,1.0,0.0,1.000000,19.889999,1,NaN,NaN,NaN,NaN,NaN,NaN
300926,43,284,2,2,0.0,0.000000,27.264999,28.015499,28.015499,28.015499,90.0,90.0,8.108871e+05,59,600,37,mixed,H,P,998,730,RU,16,0,0,28.015499,27.264999,0,0,2,0,129917,10,38.0,0.0,1.500000,27.493000,4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348857,48,2657,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.000000e+00,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12,0.0,0.0,0.000000,NaN,7,0.0,0.0,0.0,0.0,0.0,0.0
356468,49,2657,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.000000e+00,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12,0.0,0.0,0.000000,NaN,7,0.0,0.0,0.0,0.0,0.0,0.0
364174,50,2657,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.000000e+00,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12,0.0,0.0,0.000000,NaN,7,0.0,0.0,0.0,0.0,0.0,0.0
371873,51,2657,15,0,0.0,0.000000,NaN,NaN,NaN,NaN,78.0,78.0,0.000000e+00,4,400,18,traffic,H,P,19398,730,RU,0,0,0,NaN,NaN,0,0,0,0,0,12,0.0,0.0,0.000000,NaN,7,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 1. Prepare data
# Add lags of sales
all_data = all_data.sort_values(['index_material', 'index_store', 'week_iso'])
for lag in [1, 2, 3]:
    all_data[f'qnt_lag_{lag}'] = all_data.groupby(['index_material', 'index_store'])['qnt'].shift(lag)
    all_data[f'action_price_lag_{lag}'] = all_data.groupby(['index_material', 'index_store'])['action_price'].shift(lag)

# Deal with nans
all_data = all_data.dropna(subset=[f'qnt_lag_{i}' for i in range(1, 4)])
for lag in [1, 2, 3]:
    all_data[f'qnt_lag_{lag}'] = all_data[f'qnt_lag_{lag}'].astype('int16')
all_data = all_data.sort_values(by='week_iso')

<ipython-input-43-020fbe285cb3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data[f'qnt_lag_{lag}'] = all_data[f'qnt_lag_{lag}'].astype('int16')
<ipython-input-43-020fbe285cb3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data[f'qnt_lag_{lag}'] = all_data[f'qnt_lag_{lag}'].astype('int16')
<ipython-input-43-020fbe285cb3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [ ]:
# Fill NaN values with 0 for specified columns
cols_to_fillna = [
    'cluster_other_price_mean_lag1',
    'cluster_other_price_mean_lag2',
    'cluster_other_price_mean_lag3',
    'action_price_lag_1',
    'action_price_lag_2',
    'action_price_lag_3'
]

all_data[cols_to_fillna] = all_data[cols_to_fillna].fillna(0)

# Convert relevant columns to int type
cols_to_int = [
    'cluster_qnt_other_sum',
    'cluster_other_items_count',  # Assuming this represents a count of items
    'cluster_other_qnt_sum_lag1',
    'cluster_other_qnt_sum_lag2',
    'cluster_other_qnt_sum_lag3'
]

for col in cols_to_int:
    if col in all_data.columns: #Check if the column exists before converting to int
        all_data[col] = all_data[col].astype('int16')
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349238 entries, 22485 to 379536
Data columns (total 50 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   week_iso                       349238 non-null  int8    
 1   index_material                 349238 non-null  int16   
 2   index_store                    349238 non-null  int8    
 3   qnt                            349238 non-null  int16   
 4   qnt_loss                       349238 non-null  float32 
 5   discount                       349238 non-null  float32 
 6   action_mediana_price           310724 non-null  float32 
 7   reg_mediana_price              310724 non-null  float32 
 8   regular_price                  347313 non-null  float32 
 9   action_price                   347313 non-null  float32 
 10  netto                          349238 non-null  float32 
 11  brutto                         349238 non-null  float32 
 12  sum_action_price_

### LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [ ]:
train, val, test = split_data(all_data)

In [ ]:
# 2. Features (exclude only target)
features = [col for col in train.columns if col not in ['qnt']]
cat_features = ['geolocal_type', 'format_merch', 'matrix_type', 'country_id']

# 3. Category features for LightGBM
for col in cat_features:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

<ipython-input-47-9b7063c8676a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col] = train[col].astype('category')
<ipython-input-47-9b7063c8676a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[col] = test[col].astype('category')


In [ ]:
# 4. Prepare data splits
X_train = train[features]
y_train = train['qnt']
X_test = test[features]
y_test = test['qnt']

# 5. Initialize LightGBM with best parameters
best_model = lgb.LGBMRegressor(
    random_state=42,
    verbose=-1,
    objective='mae',
    force_col_wise=True,
    learning_rate=0.1,
    max_depth=8,
    n_estimators=200,
    num_leaves=127,
    reg_lambda=3
)

# 6. Train the model
best_model.fit(
    X_train,
    y_train,
    categorical_feature=cat_features
)

LGBMRegressor(force_col_wise=True, max_depth=8, n_estimators=200,
              num_leaves=127, objective='mae', random_state=42, reg_lambda=3,
              verbose=-1)

In [ ]:
X_val = val[features]
y_val = val['qnt']

# 7. Function for prediction
def predict(model, X):
    y_pred = model.predict(X)
    y_pred = np.ceil(y_pred)
    return np.maximum(y_pred, 0)

# 8. Forecasts
y_train_pred = predict(best_model, X_train)
y_val_pred = predict(best_model, X_val)
y_test_pred = predict(best_model, X_test)

# 9. Metrics calculation functions
def print_metrics(y_true, y_pred, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    median_ape = 100 * np.median(np.abs((y_true - y_pred) / (y_true + 1e-10)))

    print(f"\n{name} Metrics:")
    print(f"MAE: {mae:.2f} units")
    print(f"RMSE: {rmse:.2f} units")
    print(f"Median APE: {median_ape:.2f}%")
    print(f"Sample size: {len(y_true)}")

# 10. Вывод метрик для всех наборов
print_metrics(y_train, y_train_pred, "Train")
print_metrics(y_val, y_val_pred, "Validation")
print_metrics(y_test, y_test_pred, "Test")


Train Metrics:
MAE: 12.65 units
RMSE: 24.64 units
Median APE: 30.00%
Sample size: 304008

Validation Metrics:
MAE: 14.26 units
RMSE: 25.19 units
Median APE: 33.33%
Sample size: 22498

Test Metrics:
MAE: 13.67 units
RMSE: 25.14 units
Median APE: 32.83%
Sample size: 22732


# extra LSTM

LSTM model was also cheched during our research. It was also checked for the subcat 420201. It did not provide the best results.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Parameters
HISTORY_SIZE = 12    # number of past weeks to use
HORIZON = 3          # predict 3 weeks ahead
BATCH_SIZE = 64
EPOCHS = 30
PATIENCE = 5

# ----------------------------------------------------------------------------
# 1. Dealing with missing values
# ----------------------------------------------------------------------------
# Fill numeric columns with median and categoricals with mode
data = df.copy()
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = ['geolocal_type', 'format_merch', 'matrix_type', 'country_id']

# Impute
for col in numeric_cols:
    data[col].fillna(data[col].median(), inplace=True)
for col in cat_cols:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Encode categorical features
for col in cat_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))

# Sort by group and time
data = data.sort_values(['index_material', 'index_store', 'week_iso']).reset_index(drop=True)

# Features and target
target_col = 'qnt'
feature_cols = [c for c in data.columns if c not in [target_col, 'week_iso']]

# Verify no NaNs before scaling
assert not data[feature_cols].isna().any().any(), "NaNs detected in features after imputation."

# Scale numeric features
scaler = StandardScaler()
data[feature_cols] = scaler.fit_transform(data[feature_cols])

# Add saving scaler for target variable
target_scaler = StandardScaler()
data[[target_col]] = target_scaler.fit_transform(data[[target_col]])

# ----------------------------------------------------------------------------
# 2. Sequence generation from full dataset
# ----------------------------------------------------------------------------
sequences = []
for _, grp in data.groupby(['index_material', 'index_store']):
    grp = grp.sort_values('week_iso')
    feats = grp[feature_cols].values
    targets = grp[target_col].values
    weeks = grp['week_iso'].values
    # Generate sliding windows for each group
    n_rows = len(grp)
    if n_rows < HISTORY_SIZE + HORIZON:
        continue
    for i in range(n_rows - HISTORY_SIZE - HORIZON + 1):
        X_win = feats[i:i + HISTORY_SIZE]
        y_win = targets[i + HISTORY_SIZE:i + HISTORY_SIZE + HORIZON]
        pred_week = weeks[i + HISTORY_SIZE]  # first forecast week
        sequences.append((X_win, y_win, pred_week))

# Convert into arrays
X_all = np.array([s[0] for s in sequences])  # shape: (n_samples, HISTORY_SIZE, n_features)
y_all = np.array([s[1] for s in sequences])  # shape: (n_samples, HORIZON)
pred_weeks = np.array([s[2] for s in sequences])

# ----------------------------------------------------------------------------
# 3. Train/val/test split by pred_weeks
# ----------------------------------------------------------------------------
weeks_sorted = np.sort(data['week_iso'].unique())
train_weeks = weeks_sorted[:-2 * HORIZON]
val_weeks   = weeks_sorted[-2 * HORIZON:-HORIZON]
test_weeks  = weeks_sorted[-HORIZON:]

train_mask = np.isin(pred_weeks, train_weeks)
val_mask   = np.isin(pred_weeks, val_weeks)
test_mask  = np.isin(pred_weeks, test_weeks)

X_train, y_train = X_all[train_mask], y_all[train_mask]
X_val,   y_val   = X_all[val_mask],   y_all[val_mask]
X_test,  y_test  = X_all[test_mask],  y_all[test_mask]

print(f"Train samples: {X_train.shape[0]}, Val samples: {X_val.shape[0]}, Test samples: {X_test.shape[0]}")

# ----------------------------------------------------------------------------
# 4. Build LSTM model with explicit Input layer
# ----------------------------------------------------------------------------
n_features = len(feature_cols)
inputs = Input(shape=(HISTORY_SIZE, n_features))
x = LSTM(64)(inputs)
outputs = Dense(HORIZON)(x)
model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='mae')
model.summary()

# ----------------------------------------------------------------------------
# 5. Train with EarlyStopping
# ----------------------------------------------------------------------------
es = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es]
)
# ----------------------------------------------------------------------------
# 6. Evaluate and print metrics (with post-processing)
# ----------------------------------------------------------------------------

def postprocess_predictions(pred):
    pred = np.ceil(pred)  # round the forecast
    pred[pred < 0] = 0    # negative values replace with 0
    return pred

# Get forecast
y_pred_val = model.predict(X_val)
y_pred_test = model.predict(X_test)

# Inverse transformation and post-processing
y_val_inv = target_scaler.inverse_transform(y_val.reshape(-1, 1)).flatten()
y_pred_val_inv = postprocess_predictions(target_scaler.inverse_transform(y_pred_val.reshape(-1, 1))).flatten()

y_test_inv = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_pred_test_inv = postprocess_predictions(target_scaler.inverse_transform(y_pred_test.reshape(-1, 1))).flatten()

# Metrics calculation
val_mae = mean_absolute_error(y_val_inv, y_pred_val_inv)
val_rmse = np.sqrt(mean_squared_error(y_val_inv, y_pred_val_inv))

test_mae = mean_absolute_error(y_test_inv, y_pred_test_inv)
test_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_test_inv))

print(f"Validation MAE (post-processed): {val_mae:.4f}, RMSE (post-processed): {val_rmse:.4f}")
print(f"Test MAE (post-processed): {test_mae:.4f}, RMSE (post-processed): {test_rmse:.4f}")

<ipython-input-25-77b4f8c74398>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
<ipython-input-25-77b4f8c74398>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

Train samples: 220024, Val samples: 20522, Test samples: 7043


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 12, 30)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 64)             │        24,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,515 (95.76 KB)

 Trainable params: 24,515 (95.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - loss: 0.3994 - val_loss: 0.3801
Epoch 2/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.3288 - val_loss: 0.3616
Epoch 3/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 42s 11ms/step - loss: 0.3074 - val_loss: 0.3429
Epoch 4/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 37s 11ms/step - loss: 0.2965 - val_loss: 0.3434
Epoch 5/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - loss: 0.2897 - val_loss: 0.3329
Epoch 6/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 42s 11ms/step - loss: 0.2838 - val_loss: 0.3651
Epoch 7/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 41s 11ms/step - loss: 0.2789 - val_loss: 0.3357
Epoch 8/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.2755 - val_loss: 0.3445
Epoch 9/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 42s 11ms/step - loss: 0.2704 - val_loss: 0.3336
Epoch 10/30
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.2685 - val_loss: 0.3334
642/642 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
221/221 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Validation MAE (p

# Subcat 380101

There is a weekly data for subcategory 380101 of products. It is used to check if the model that takes into account the cannibalization effect provides better forecasts than the model without cannibalization not only for one category but can be applied to others as well.

In [9]:
df = pd.read_parquet('/content/drive/MyDrive/weekly_data_380101.parquet', engine='pyarrow')
df

,week_iso,index_material,index_store,qnt,qnt_loss,discount,action_mediana_price,reg_mediana_price,regular_price,action_price,netto,brutto,sum_action_price_by_brand,am_items_by_brand,count_park,count_cashzone,geolocal_type,format_merch,matrix_type,brand_id,shelf,country_id,type_for_customer,regular_pnz,promo_pnz,last_reg_mediana_price,last_action_mediana_price,type_bonus_id,bu_exists,flg_promo,is_holiday,qnt_by_brand
0,1,89,0,0,0.0,0.127154,NaN,NaN,19.990000,17.448181,144.0,144.0,60594.875000,8,850,37,traffic,H,A,4488,365,RU,0,0,0,20.03550,19.990000,1,0,4,7,1691
1,1,89,1,36,0.0,0.127557,13.075714,20.035500,20.023090,17.468864,144.0,144.0,60594.875000,8,700,43,mixed,H,A,4488,365,RU,0,0,0,20.03550,12.399796,1,0,4,7,1691
2,1,89,2,10,0.0,0.127289,10.710000,20.035500,20.006544,17.460590,144.0,144.0,60594.875000,8,600,37,mixed,H,A,4488,365,RU,0,0,0,20.03550,10.710000,1,0,4,7,1691
3,1,89,3,19,0.0,0.116135,16.844000,19.940500,19.927876,17.613543,144.0,144.0,67202.171875,8,1100,36,traffic,H,A,4488,365,RU,9,0,0,19.94050,16.201250,1,0,4,7,1691
4,1,89,4,21,0.0,0.127423,14.918030,20.019667,20.006182,17.456091,144.0,144.0,60594.875000,8,450,36,mixed,H,A,4488,365,RU,9,8,0,20.00959,14.959654,1,0,4,7,1691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190625,52,2591,45,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,842009.687500,22,200,6,mixed,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038
190626,52,2591,46,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,421004.843750,22,0,8,urban,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038
190627,52,2591,47,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,421004.843750,22,100,9,mixed,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038
190628,52,2591,49,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,421004.843750,22,100,9,mixed,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038


In [ ]:
df.index_material.nunique()

145

In [ ]:
df.index_store.nunique()

83

In [ ]:
df.brand_id.nunique()

15

So, there are 145 unique products, 83 different shops and 15 brands in this subcategory.

## Baseline model **without** cannibalization effect for subcategory 380101

In [10]:
# Splitting data on train/val/test (3 weeks in val and test)
def split_data(df):
    weeks = df['week_iso'].unique()
    weeks.sort()

    # Last 6 weeks - 3 for val and 3 for test
    val_weeks = weeks[-3:]
    test_weeks = weeks[-6:-3]
    train_weeks = weeks[:-6]

    # Разделяем данные
    train = df[df['week_iso'].isin(train_weeks)]
    val = df[df['week_iso'].isin(val_weeks)]
    test = df[df['week_iso'].isin(test_weeks)]

    return train, val, test

In [11]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [12]:
# 1. Function to create lag features
def add_lags(df, lags=3):
    """Add lagged sales quantities for each product-store combination"""
    df = df.sort_values(['index_material', 'index_store', 'week_iso'])
    for lag in range(1, lags+1):
        df[f'qnt_lag_{lag}'] = df.groupby(['index_material', 'index_store'])['qnt'].shift(lag)
    return df

# 2. Add lag features to all datasets
df = add_lags(df.copy())
train, val, test = split_data(df)

# 3. Define feature types and target variable
target = 'qnt'
exclude_cols = ['week_iso', 'index_material', 'index_store', target]

# Numeric features (excluding lags which are already processed)
numeric_features = train.select_dtypes(include=['int16', 'int32', 'int64', 'float32', 'float64']).columns
numeric_features = [col for col in numeric_features if col not in exclude_cols and not col.startswith('qnt_lag_')]

# Categorical features
categorical_features = train.select_dtypes(include=['category', 'object']).columns.tolist()

# Sales lag features (already numeric)
lag_features = [f'qnt_lag_{i}' for i in range(1, 4)]

# All features to be used in model
all_features = numeric_features + categorical_features + lag_features

# 4. Create feature preprocessor with imputation
preprocessor = ColumnTransformer(
    transformers=[
        # Scale numeric features and lags (with imputation)
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features + lag_features),
        # One-hot encode categorical features (with imputation)
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

# 5. Create modeling pipeline
model = Pipeline([
    ('preprocessor', preprocessor),  # Feature preprocessing
    ('regressor', LinearRegression())  # Linear regression model
])

# 6. Prepare data (drop rows where target is NA)
train = train.dropna(subset=[target])
val = val.dropna(subset=[target])
test = test.dropna(subset=[target])

X_train, y_train = train[all_features], train[target]
X_val, y_val = val[all_features], val[target]
X_test, y_test = test[all_features], test[target]

# 7. Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['qnt_loss', 'discount',
                                                   'action_mediana_price',
                                                   'reg_mediana_price',
                                                   'regular_price',
                                                   'action_price', 'netto',
                                                   'brutto',
                                                   'sum_action_price_by_brand',
                                                   'count_park',
                                                   'count_cashzone', 'brand_id',
                                                   'shel..._pnz',
                                                   'promo_pnz',
                                                   'last_reg_mediana_price',
                                                   'last_action_mediana_price',
                                                   'qnt_by_brand', 'qnt_lag_1',
                                                   'qnt_lag_2', 'qnt_lag_3']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['geolocal_type',
                                                   'format_merch',
                                                   'matrix_type',
                                                   'country_id'])])),
                ('regressor', LinearRegression())])

In [13]:
# 8. Prediction function with post-processing
def predict(model, X):
    """Make predictions and apply post-processing"""
    y_pred = model.predict(X)
    y_pred = np.ceil(y_pred)  # Round up to nearest integer
    return np.maximum(y_pred, 0)  # Replace negatives with 0

# 9. Generate predictions
y_train_pred = predict(model, X_train)
y_val_pred = predict(model, X_val)
y_test_pred = predict(model, X_test)

# 10. Evaluation metrics
def print_metrics(y_true, y_pred, name):
    """Calculate and print evaluation metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Median Absolute Percentage Error (robust to zeros)
    median_ape = 100 * np.median(np.abs((y_true - y_pred) / (y_true + 1e-10)))

    print(f"\n{name} Metrics:")
    print(f"MAE: {mae:.2f} units")
    print(f"RMSE: {rmse:.2f} units")
    print(f"Median APE: {median_ape:.2f}%")

# Print metrics for all datasets
print_metrics(y_train, y_train_pred, "Train")
print_metrics(y_val, y_val_pred, "Validation")
print_metrics(y_test, y_test_pred, "Test")


Train Metrics:
MAE: 5.37 units
RMSE: 9.97 units
Median APE: 50.00%

Validation Metrics:
MAE: 6.38 units
RMSE: 20.39 units
Median APE: 50.00%

Test Metrics:
MAE: 6.19 units
RMSE: 19.07 units
Median APE: 50.00%


## The model **with** the cannibalization effect for subcategory 420201

### OPTICS with Euclidean distance

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score
from sklearn.cluster import OPTICS
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [ ]:
def optics(data):

    # 1. Data aggregation for products
    product_features = data.groupby('index_material').agg({
        'qnt': 'mean',
        'action_mediana_price': 'mean',
        'reg_mediana_price': 'mean',
        'regular_price': 'mean',
        'action_price': 'mean',
        'netto': 'mean',
        'brutto': 'mean',
        'brand_id':'mean',
        'shelf':'mean',
        'sum_action_price_by_brand': 'mean',
        'am_items_by_brand': 'mean',
        'matrix_type': lambda x: x.mode()[0] if not x.mode().empty else pd.NA,
        'type_for_customer': lambda x: x.mode()[0] if not x.mode().empty else pd.NA,
        'bu_exists': lambda x: x.mode()[0] if not x.mode().empty else pd.NA
    }).reset_index()

    # 2. Handling missing values
    # For numeric features
    num_cols = ['qnt', 'action_mediana_price', 'reg_mediana_price',
               'regular_price', 'action_price', 'netto', 'brutto',
               'sum_action_price_by_brand', 'am_items_by_brand']
    product_features[num_cols] = product_features[num_cols].fillna(0)

    # For categories
    cat_cols = ['matrix_type', 'type_for_customer', 'bu_exists']
    for col in cat_cols:
        product_features[col] = product_features[col].astype(str).fillna('missing')
        product_features[col] = product_features[col].astype('category').cat.codes

    # 3. Scaler
    scaler = StandardScaler()
    X = scaler.fit_transform(product_features.drop('index_material', axis=1))

    # 4. Extra missing values handling
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    # 5. Clustering
    optics = OPTICS(
        min_samples=max(5, int(0.01*len(product_features))),  # Auto
        metric='euclidean',
        algorithm='ball_tree',
        n_jobs=-1,
        cluster_method='xi'
    )

    product_features['cluster'] = optics.fit_predict(X)

    # Silhouette score calculation
    unique_clusters = np.unique(product_features['cluster'])
    if len(unique_clusters) > 1:
        silhouette_avg = silhouette_score(X, product_features['cluster'])
        print(f"Silhouette Score: {silhouette_avg:.3f}")
    else:
        print("Silhouette score can not be calculated for 1 cluster")
        silhouette_avg = None

    # 6. Add cluster to the original data
    return data.merge(
        product_features[['index_material', 'cluster']],
        on='index_material',
        how='left'
    )['cluster'].fillna(-1).astype(int)

In [ ]:
# Apply
all_data = df.copy()
all_data['cluster'] = optics(all_data)
all_data

Silhouette Score: 0.382


,week_iso,index_material,index_store,qnt,qnt_loss,discount,action_mediana_price,reg_mediana_price,regular_price,action_price,netto,brutto,sum_action_price_by_brand,am_items_by_brand,count_park,count_cashzone,geolocal_type,format_merch,matrix_type,brand_id,shelf,country_id,type_for_customer,regular_pnz,promo_pnz,last_reg_mediana_price,last_action_mediana_price,type_bonus_id,bu_exists,flg_promo,is_holiday,qnt_by_brand,cluster
0,1,89,0,0,0.0,0.127154,NaN,NaN,19.990000,17.448181,144.0,144.0,60594.875000,8,850,37,traffic,H,A,4488,365,RU,0,0,0,20.03550,19.990000,1,0,4,7,1691,0
1,1,89,1,36,0.0,0.127557,13.075714,20.035500,20.023090,17.468864,144.0,144.0,60594.875000,8,700,43,mixed,H,A,4488,365,RU,0,0,0,20.03550,12.399796,1,0,4,7,1691,0
2,1,89,2,10,0.0,0.127289,10.710000,20.035500,20.006544,17.460590,144.0,144.0,60594.875000,8,600,37,mixed,H,A,4488,365,RU,0,0,0,20.03550,10.710000,1,0,4,7,1691,0
3,1,89,3,19,0.0,0.116135,16.844000,19.940500,19.927876,17.613543,144.0,144.0,67202.171875,8,1100,36,traffic,H,A,4488,365,RU,9,0,0,19.94050,16.201250,1,0,4,7,1691,0
4,1,89,4,21,0.0,0.127423,14.918030,20.019667,20.006182,17.456091,144.0,144.0,60594.875000,8,450,36,mixed,H,A,4488,365,RU,9,8,0,20.00959,14.959654,1,0,4,7,1691,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190625,52,2591,45,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,842009.687500,22,200,6,mixed,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038,-1
190626,52,2591,46,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,421004.843750,22,0,8,urban,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038,-1
190627,52,2591,47,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,421004.843750,22,100,9,mixed,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038,-1
190628,52,2591,49,0,0.0,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,421004.843750,22,100,9,mixed,S,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,9038,-1


In [ ]:
all_data.cluster.value_counts()

,count
cluster,
-1,37133
7,32068
8,18405
2,17980
3,16049
4,15507
0,14403
6,10508
15,7643


There are 17 clusters.

### Adding cannibalization features

In [ ]:
# Function to catch cannibalization effect
def add_cannibalization_features(df, lags=[1,2,3]):
    """
    Adds cannibalization features excluding the current product from the aggregation
    """
    # Create a copy without current product for each line
    df_list = []
    for idx, row in df.iterrows():
        # Filter: the same cluster and shop but not the same product
        cluster_mates = df[
            (df['cluster'] == row['cluster']) &
            (df['index_store'] == row['index_store']) &
            (df['index_material'] != row['index_material']) &
            (df['week_iso'] == row['week_iso'])
        ]

        # Aggregate
        agg_stats = cluster_mates.agg({
            'qnt': ['sum', 'median'],
            'action_price': ['mean', 'median']
        }).to_numpy().flatten()

        new_row = {
            'cluster_qnt_other_sum': agg_stats[0],
            'cluster_qnt_other_median': agg_stats[1],
            'cluster_price_other_mean': agg_stats[2],
            'cluster_price_other_median': agg_stats[3],
            'cluster_other_items_count': len(cluster_mates)
        }
        df_list.append(new_row)

    # Merge with the original data
    cannib_stats = pd.DataFrame(df_list, index=df.index)
    df = pd.concat([df, cannib_stats], axis=1)

    # Add lags
    df = df.sort_values(['index_material', 'index_store', 'week_iso'])
    for lag in lags:
        # Lags of aggregated features of outher products
        df[f'cluster_other_qnt_sum_lag{lag}'] = df.groupby(
            ['index_material', 'index_store']
        )['cluster_qnt_other_sum'].shift(lag)

        df[f'cluster_other_price_mean_lag{lag}'] = df.groupby(
            ['index_material', 'index_store']
        )['cluster_price_other_mean'].shift(lag)

    # Fill the missing values
    df.fillna({
        'cluster_qnt_other_sum': 0,
        'cluster_qnt_other_median': 0,
        'cluster_price_other_mean': df['action_price'],  # If there are no other products use its price
        'cluster_price_other_median': df['action_price'],
        'cluster_other_items_count': 0
    }, inplace=True)

    return df

In [ ]:
all_data = add_cannibalization_features(all_data)
all_data

,week_iso,index_material,index_store,qnt,qnt_loss,discount,action_mediana_price,reg_mediana_price,regular_price,action_price,netto,brutto,sum_action_price_by_brand,am_items_by_brand,count_park,count_cashzone,geolocal_type,format_merch,matrix_type,brand_id,shelf,country_id,type_for_customer,regular_pnz,promo_pnz,last_reg_mediana_price,last_action_mediana_price,type_bonus_id,bu_exists,flg_promo,is_holiday,qnt_by_brand,cluster,cluster_qnt_other_sum,cluster_qnt_other_median,cluster_price_other_mean,cluster_price_other_median,cluster_other_items_count,cluster_other_qnt_sum_lag1,cluster_other_price_mean_lag1,cluster_other_qnt_sum_lag2,cluster_other_price_mean_lag2,cluster_other_qnt_sum_lag3,cluster_other_price_mean_lag3
0,1,89,0,0,0.00000,0.127154,NaN,NaN,19.990000,17.448181,144.0,144.0,60594.875000,8,850,37,traffic,H,A,4488,365,RU,0,0,0,20.035500,19.990000,1,0,4,7,1691,0,8.0,0.0,0.0,16.559135,5,NaN,NaN,NaN,NaN,NaN,NaN
3217,2,89,0,2,0.00000,0.043762,19.990000,20.035500,19.995687,19.206608,144.0,144.0,45356.062500,8,850,37,traffic,H,A,4488,365,RU,0,0,0,20.035500,19.990000,1,0,1,0,1831,0,2.0,0.0,0.0,17.856184,5,8.0,0.0,NaN,NaN,NaN,NaN
6501,3,89,0,7,0.00000,0.094822,18.564999,20.676750,20.755154,18.774307,144.0,144.0,80074.242188,8,850,37,traffic,H,A,4488,365,RU,0,0,0,20.364346,19.633749,1,0,5,0,2284,0,20.0,0.0,3.0,18.698416,5,2.0,0.0,8.0,0.0,NaN,NaN
9767,4,89,0,17,0.00000,0.122374,16.230000,20.890499,20.890312,18.333874,144.0,144.0,104340.632812,8,850,37,traffic,H,A,4488,365,RU,1,14,0,20.890499,17.816668,1,0,7,0,2627,0,40.0,0.0,6.0,18.436968,5,20.0,3.0,2.0,0.0,8.0,0.0
13000,5,89,0,5,0.01322,0.072245,18.090000,20.646214,20.571226,19.061998,144.0,144.0,70753.664062,8,850,37,traffic,H,A,4488,365,RU,1,0,0,20.735044,18.362726,1,0,3,0,2233,0,57.0,0.0,10.0,18.615101,5,40.0,6.0,20.0,3.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174904,48,2591,60,0,0.00000,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,167708.218750,22,0,4,urban,C,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,7156,-1,12.0,0.0,1.0,45.474907,6,NaN,NaN,NaN,NaN,NaN,NaN
178847,49,2591,60,0,0.00000,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,323001.750000,22,0,4,urban,C,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,6995,-1,6.0,0.0,0.0,49.754795,7,12.0,1.0,NaN,NaN,NaN,NaN
182848,50,2591,60,0,0.00000,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,344347.562500,22,0,4,urban,C,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,7524,-1,17.0,0.0,2.0,47.411133,7,6.0,0.0,12.0,1.0,NaN,NaN
186766,51,2591,60,0,0.00000,0.000000,NaN,NaN,NaN,NaN,23300.0,23300.0,301364.812500,22,0,4,urban,C,A,3336,9999,RU,0,0,0,NaN,NaN,0,0,0,0,7836,-1,13.0,0.0,2.0,45.884766,7,17.0,2.0,6.0,0.0,12.0,1.0


In [ ]:
# 1. Prepare data
# Add lags of sales
all_data = all_data.sort_values(['index_material', 'index_store', 'week_iso'])
for lag in [1, 2, 3]:
    all_data[f'qnt_lag_{lag}'] = all_data.groupby(['index_material', 'index_store'])['qnt'].shift(lag)
    all_data[f'action_price_lag_{lag}'] = all_data.groupby(['index_material', 'index_store'])['action_price'].shift(lag)

# Deal with nans
all_data = all_data.dropna(subset=[f'qnt_lag_{i}' for i in range(1, 4)])
for lag in [1, 2, 3]:
    all_data[f'qnt_lag_{lag}'] = all_data[f'qnt_lag_{lag}'].astype('int16')
all_data = all_data.sort_values(by='week_iso')

<ipython-input-30-020fbe285cb3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data[f'qnt_lag_{lag}'] = all_data[f'qnt_lag_{lag}'].astype('int16')
<ipython-input-30-020fbe285cb3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data[f'qnt_lag_{lag}'] = all_data[f'qnt_lag_{lag}'].astype('int16')
<ipython-input-30-020fbe285cb3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [ ]:
# Fill NaN values with 0 for specified columns
cols_to_fillna = [
    'cluster_other_price_mean_lag1',
    'cluster_other_price_mean_lag2',
    'cluster_other_price_mean_lag3',
    'action_price_lag_1',
    'action_price_lag_2',
    'action_price_lag_3'
]

all_data[cols_to_fillna] = all_data[cols_to_fillna].fillna(0)

# Convert relevant columns to int type
cols_to_int = [
    'cluster_qnt_other_sum',
    'cluster_other_items_count',  # Assuming this represents a count of items
    'cluster_other_qnt_sum_lag1',
    'cluster_other_qnt_sum_lag2',
    'cluster_other_qnt_sum_lag3'
]

for col in cols_to_int:
    if col in all_data.columns: #Check if the column exists before converting to int
        all_data[col] = all_data[col].astype('int16')
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174061 entries, 9767 to 190629
Data columns (total 50 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   week_iso                       174061 non-null  int8    
 1   index_material                 174061 non-null  int16   
 2   index_store                    174061 non-null  int8    
 3   qnt                            174061 non-null  int16   
 4   qnt_loss                       174061 non-null  float32 
 5   discount                       174061 non-null  float32 
 6   action_mediana_price           141501 non-null  float32 
 7   reg_mediana_price              141501 non-null  float32 
 8   regular_price                  173959 non-null  float32 
 9   action_price                   173959 non-null  float32 
 10  netto                          174061 non-null  float32 
 11  brutto                         174061 non-null  float32 
 12  sum_action_price_b

### LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [ ]:
train, val, test = split_data(all_data)

In [ ]:
# 2. Features (exclude only target)
features = [col for col in train.columns if col not in ['qnt']]
cat_features = ['geolocal_type', 'format_merch', 'matrix_type', 'country_id']

# 3. Category features for LightGBM
for col in cat_features:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

<ipython-input-35-9b7063c8676a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col] = train[col].astype('category')
<ipython-input-35-9b7063c8676a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[col] = test[col].astype('category')


In [ ]:
# 4. Prepare data splits
X_train = train[features]
y_train = train['qnt']
X_test = test[features]
y_test = test['qnt']

# 5. Initialize LightGBM with best parameters
best_model = lgb.LGBMRegressor(
    random_state=42,
    verbose=-1,
    objective='mae',
    force_col_wise=True,
    learning_rate=0.1,
    max_depth=8,
    n_estimators=200,
    num_leaves=127,
    reg_lambda=3
)

# 6. Train the model
best_model.fit(
    X_train,
    y_train,
    categorical_feature=cat_features
)

LGBMRegressor(force_col_wise=True, max_depth=8, n_estimators=200,
              num_leaves=127, objective='mae', random_state=42, reg_lambda=3,
              verbose=-1)

In [ ]:
X_val = val[features]
y_val = val['qnt']

# 7. Function for prediction
def predict(model, X):
    y_pred = model.predict(X)
    y_pred = np.ceil(y_pred)
    return np.maximum(y_pred, 0)

# 8. Forecasts
y_train_pred = predict(best_model, X_train)
y_val_pred = predict(best_model, X_val)
y_test_pred = predict(best_model, X_test)

# 9. Metrics calculation functions
def print_metrics(y_true, y_pred, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    median_ape = 100 * np.median(np.abs((y_true - y_pred) / (y_true + 1e-10)))

    print(f"\n{name} Metrics:")
    print(f"MAE: {mae:.2f} units")
    print(f"RMSE: {rmse:.2f} units")
    print(f"Median APE: {median_ape:.2f}%")
    print(f"Sample size: {len(y_true)}")

# 10. Вывод метрик для всех наборов
print_metrics(y_train, y_train_pred, "Train")
print_metrics(y_val, y_val_pred, "Validation")
print_metrics(y_test, y_test_pred, "Test")


Train Metrics:
MAE: 3.64 units
RMSE: 7.47 units
Median APE: 38.46%
Sample size: 150760

Validation Metrics:
MAE: 4.31 units
RMSE: 9.18 units
Median APE: 50.00%
Sample size: 11636

Test Metrics:
MAE: 3.87 units
RMSE: 8.15 units
Median APE: 50.00%
Sample size: 11665


So, for another category the model with the cannibalization works good as well.